# Вывод на экран сотрудников, которым необходима замена документов в текущем года в связи с окончанием срока действия.

In [1]:
# загружаем библиотеки
import pandas as pd
from datetime import datetime, timedelta, date
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
# подключаемся к гугл диску
from google.colab import drive
drive.mount('/content/drive')
# initsyals

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# загружаем датасеты
df_1 = pd.read_excel('/content/drive/MyDrive/Работа/Свойства.xlsx', sheet_name='TDSheet') # список всех документов всех сотрудников с момента основания организации
df_milk = pd.read_excel('/content/drive/MyDrive/Работа/молоко-1.xlsx', sheet_name='TDSheet') # список водителей цистерн
df_van = pd.read_excel('/content/drive/MyDrive/Работа/фур.xlsx', sheet_name='TDSheet')       # список водителей фургонов
df_auto = pd.read_excel('/content/drive/MyDrive/Работа/прочие.xlsx', sheet_name='TDSheet')   # список прочих водителей

In [4]:
# переимоную столбцы датасета
df_milk = df_milk.rename(columns={'Фамилия, имя, отчество полностью': 'initsyals', 'Должность': 'post'})
df_van = df_van.rename(columns={'Фамилия, имя, отчество полностью': 'initsyals', 'Должность': 'post'})
df_auto = df_auto.rename(columns={'Фамилия, имя, отчество полностью': 'initsyals', 'Должность': 'post'})

In [5]:
#  собираю ФИО сотрудников в одном списке
initsyals_list = []
data = [df_milk, df_van, df_auto]
for i in data:
  initsyals_list.extend(i['initsyals'].unique())


In [6]:
# удаляю строки с лишними данными
propertys = ['Разряд', 'Тип молоковоза', 'Серия и № паспорта', 'Топливная карта', 'Без СКЗИ']
for i in propertys:
  df_1 = df_1.drop(df_1[df_1['Свойство'] == i].index)
  df_1 = df_1.drop(df_1[df_1['Тип карты'] == i].index)
df_1= df_1.rename(columns={'Дата начала': 'data', 'Сотрудник':'employee', 'Свойство':'property', 'Значение':'values', 'Дата окончания':'date_end', 'Тип карты':'card_type'})


In [7]:
# привожу даты к нужному формату и выделяю год
df_1 = df_1.fillna(0)
df_1['data'] = pd.to_datetime(df_1['data']).dt.date
df_1['date_end'] = pd.to_datetime(df_1['date_end']).dt.date
df_1['year'] = pd.DatetimeIndex(df_1['date_end']).year

In [8]:
# делаю срез данных по видам дукументов
df_vu = df_1.loc[df_1.loc[: , 'property']=='№ водительского удостоверения'].drop(['card_type'], axis=1)
df_card = df_1.loc[df_1.loc[: , 'property']!='№ водительского удостоверения']


In [10]:
# сохраняю текущую дату и год
current_date = date.today()
current_year = current_date.year



2022

In [11]:
# функция выбирает сотрудников по ФИО из списка
# сравнивает дату окончания с текущей датой
# и если год окончания равен текущему году, то выводит на экран
# также проверяет наличие всех ФИО из списков сотрудников в списке со свойствами сотрудников
# если есть расхождения, то выводит их на экран 
def df_share(df):
    df_new = df.query('employee in @initsyals_list')
    df_new=df_new.loc[df_new.loc[: , 'date_end']>current_date].reset_index(drop=True)
    df_new_1 = df_new['employee'].unique()
    df_new_2 = df_van.query('initsyals not in @df_new_1')
    df_2_new_1 = df_milk.query('initsyals not in @df_new_1')
    df_2_new_2 = df_auto.query('initsyals not in @df_new_1')
    print('Требуется замена в текущем году')
    df_new_3 = df_new.loc[df_new.loc[: , 'year'] == current_year]
    
    return df_new_3, df_new_2, df_2_new_1, df_2_new_2

In [12]:
# передам функции датасета с информацией о вод.удостоверении
a, b, c, d = df_share(df_vu)
display(a)
print('Отсутствуют сведения о сроке действия')
display(b)
display(c)
display(d)

Требуется замена в текущем году


,data,employee,property,values,date_end,year
0,2012-06-27,Пескишев Сергей Анатольевич,№ водительского удостоверения,0.0,2022-06-27,2022


Отсутствуют сведения о сроке действия


,initsyals,post


,initsyals,post


,initsyals,post


In [13]:
# передам функции информацию о картах водителей
a_1, a_2, a_3, a_4 = df_share(df_card)
display(a_1)
print('Отсутствуют сведения о сроке действия')
display(a_2)

display(a_4)

Требуется замена в текущем году


,data,employee,property,values,date_end,card_type,year
0,2019-04-16,Попцов Владимир Николаевич,Карта 1,0.0,2022-06-15,С СКЗИ,2022
1,2019-06-17,Федосеев Игорь Александрович,Карта 2,0.0,2022-06-15,С СКЗИ,2022
2,2019-06-17,Решетников Виктор Александрович,Карта 2,0.0,2022-06-15,С СКЗИ,2022
3,2019-03-07,Созин Виктор Николаевич,Карта 2,0.0,2022-06-15,С СКЗИ,2022
5,2020-03-23,Лапин Артур Сергеевич,Карта 1,0.0,2022-09-19,С СКЗИ,2022


Отсутствуют сведения о сроке действия


,initsyals,post
10,Девятых Михаил Александрович,Водитель автомобиля-экспедитор
16,Катков Владимир Игоревич,Водитель автомобиля-экспедитор
18,Колесов Денис Михайлович,Водитель автомобиля-экспедитор
36,Суворов Руслан Махирович,Водитель автомобиля-экспедитор


,initsyals,post
0,Машкин Александр Николаевич,водитель автомобиля 5 разряда


# Новый раздел